## Check List 7.1. Demo - White board sweeping
* In this demo file, we will use two robots (Indy7, Panda) to sweep a white board, removing obstacles

* You need trained model to use ReachChecker
  - model/reach_svm/indy7.json
  - model/reach_svm/panda.json
  - 백업: 개인 이동식 하드디스크, 강준수

### Prepare the task scene
* Prepare Indy7 and panda, and install sweeping tool to indy. (Check release/Figs/7.1.WhiteBoardSweeping.jpg)
* Prepare "floor", "track", "box1", "box2" objects as defined in pkg.detector.aruco.marker_config.py
* The robots and "track" are installed on "floor".
* "box1" and "box2" are on "track"
<img src="../Figs/7.1.WhiteBoardSweeping.jpg" width="80%">

## set running directory to project source

In [ ]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## init combined robot config

In [ ]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0.3,-0.4,0), (0,0,np.pi/2)),
                INDY_IP),
    RobotConfig(1, RobotType.panda, ((-0.3,-0.4,0), (0,0,np.pi/2)),
                "{}/{}".format(PANDA_REPEATER_IP, PANDA_ROBOT_IP))]
              , connection_list=[False, False])

In [ ]:
from pkg.detector.aruco.marker_config import *
from pkg.detector.aruco.stereo import ArucoStereo
from pkg.detector.camera.realsense import RealSense
from pkg.detector.camera.kinect import Kinect
aruco_map = get_aruco_map()
stereo = ArucoStereo(aruco_map, [Kinect(), RealSense()])
stereo.initialize()
stereo.calibrate()

## create scene builder

In [ ]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(stereo)
s_builder.reset_reference_coord(ref_name="floor")

## detect robot and make geometry scene

In [ ]:
xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

## init planning pipeline

In [ ]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## add environment

In [ ]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.ENVIRONMENT])

In [ ]:
# add cam poles
ptems = s_builder.add_poles({"cam0": s_builder.ref_coord_inv[:3,3], "cam1":np.matmul(s_builder.ref_coord_inv, stereo.T_c12)[:3,3]},
                            color=(0.6,0.6,0.6,0.0))

In [ ]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

In [ ]:
 gscene.NAME_DICT["floor"].dims = (2,1,0.01)

In [ ]:
gscene.set_workspace_boundary( -1, 1, -0.5, 0.5, -0.1, 1.1)

## add sweep face

In [ ]:
track = s_builder.detect_and_register(item_names=["track"], level_mask=[DetectionLevel.ENVIRONMENT])["track"]
track_face = gscene.copy_from(track, new_name="track_face", collision=False)
track_face.dims = (track.dims[0]-0.22, track.dims[1]-0.12, track.dims[2])
# track.dims = (track.dims[0]-0.32, track.dims[1]-0.12, track.dims[2])

In [ ]:
gscene.update_markers_all()

## add wp

In [ ]:
TOOL_DIM = [0.06, 0.10]
TRACK_DIM = track_face.dims
TRACK_NUM = np.ceil(np.divide(TRACK_DIM[1]-TOOL_DIM[1], TOOL_DIM[1])).astype(np.int)+1
TRACK_STEP = (TRACK_DIM[1]-TOOL_DIM[1])/(TRACK_NUM-1)
WP_REF_A = -np.subtract(TRACK_DIM[:2], TOOL_DIM[:2])/2
WP_REF_B = np.array([-WP_REF_A[0], WP_REF_A[1]])
TRC_THIC = TRACK_DIM[2]
track_list = []
for i_trc in range(TRACK_NUM):
    wp1 = gscene.create_safe(GEOTYPE.BOX, "wp{}a".format(i_trc+1), "base_link", (TOOL_DIM[0]/2,TOOL_DIM[1]/2,TRC_THIC), 
                             tuple(WP_REF_A+[0,TRACK_STEP*i_trc])+(0,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    wp2 = gscene.create_safe(GEOTYPE.BOX, "wp{}b".format(i_trc+1), "base_link", (TOOL_DIM[0]/2,TOOL_DIM[1]/2,TRC_THIC), 
                             tuple(WP_REF_B+[0,TRACK_STEP*i_trc])+(0,),rpy=(0,0,0), 
                             color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False, parent="track_face")
    track_list.append((wp1, wp2))

In [ ]:
gscene.update_markers_all()

## add indy tool

In [ ]:
gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="indy0_fts", link_name="indy0_tcp",
                            center=(0,0,0.02), dims=(0.07,0.07, 0.04), rpy=(0,0,0), color=(0.8,0.8,0.8,1),
                            collision=False)
gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="indy0_fts_col", link_name="indy0_tcp",
                            center=(0,0,0.02), dims=(0.11,0.11, 0.04), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5),
                            collision=True)
gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="indy0_pole", link_name="indy0_tcp",
                            center=(0,0,0.055), dims=(0.03,0.03, 0.030), rpy=(0,0,0), color=(0.8,0.8,0.8,1),
                            collision=False)
gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="indy0_pole_col", link_name="indy0_tcp",
                            center=(0,0,0.055), dims=(0.07,0.07, 0.030), rpy=(0,0,0), color=(0.0,0.8,0.0,0.2),
                            collision=True)

gscene.create_safe(gtype=GEOTYPE.BOX, name="indy0_brushbase", link_name="indy0_tcp",
                            center=(0,0,0.0775), dims=(0.06, 0.14, 0.015), rpy=(0,0,0), color=(0.8,0.8,0.8,1),
                            collision=False)
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face", link_name="indy0_tcp",
                            center=(0,0,0.095), dims=(0.05,0.13, 0.015), rpy=(0,0,0), color=(1.0,1.0,0.94,1),
                            collision=False)
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face_col", link_name="indy0_tcp",
                            center=(0,0,0.080), dims=(0.08,0.18, 0.02), rpy=(0,0,0), color=(0.0,0.8,0.0,0.5),
                            collision=True)

## add box

In [ ]:
gtem_dict = s_builder.detect_and_register(level_mask=[DetectionLevel.MOVABLE])

In [ ]:
# give clearance from the track surface
s_builder.give_clearance(track, gtem_dict.values())

In [ ]:
gscene.update_markers_all()

## Register binders

In [ ]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepFramer, FixtureSlot

In [ ]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip1", link_name="panda1_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

pscene.create_binder(bname="grip1", gname="grip1", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="track_face", gname="track_face", _type=PlacePlane)
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepFramer, point=(0,0,0.00), 
                     rpy=(0,np.pi,0))

## add objects

In [ ]:
from pkg.planning.constraint.constraint_subject import \
        CustomObject, Grasp2Point, PlacePoint, SweepFrame, SweepTask, BoxObject, FixturePoint, AbstractObject

In [ ]:
box1 = pscene.create_subject(oname="box1", gname="box1", _type=BoxObject, hexahedral=True)
box2 = pscene.create_subject(oname="box2", gname="box2", _type=BoxObject, hexahedral=True)
# box3= pscene.create_subject(oname="box3", gname="box3", _type=BoxObject, hexahedral=True)

In [ ]:
from pkg.planning.constraint.constraint_common import MotionConstraint
from pkg.planning.constraint.constraint_subject import AbstractTask
from pkg.planning.constraint.constraint_subject import SweepLineTask

In [ ]:
sweep_list = []
for i_t, track_tem in enumerate(track_list):
    wp1, wp2 = track_tem
    sweep_ = pscene.create_subject(oname="sweep{}".format(i_t+1), gname="track_face", _type=SweepLineTask, 
                                 action_points_dict = {wp1.name: SweepFrame(wp1.name, wp1, [0,0,0.005], [0,0,0]),
                                                       wp2.name: SweepFrame(wp2.name, wp2, [0,0,0.005], [0,0,0])})
    sweep_list.append(sweep_)

### planners

In [ ]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

## motion filters

In [ ]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker

gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
checkers_all = [gcheck, rcheck]
# lcheck = LatticedChecker(pscene, gcheck)
# checkers_all.append(lcheck)

In [ ]:
mplan.motion_filters = checkers_all

In [ ]:
gscene.show_pose(crob.home_pose)

## Set initial condition

In [ ]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy
gtimer = GlobalTimer.instance()
initial_state = pscene.update_state(crob.home_pose)
print(initial_state.node)

In [ ]:
pscene.subject_name_list

# Node Sampler

In [ ]:
from pkg.planning.sampling.node_sampling import make_state_param_hashable, UniformNodeSampler, PenaltyNodeSampler, GrowingSampler

tplan.new_node_sampler = PenaltyNodeSampler(3, 1)
tplan.parent_node_sampler = UniformNodeSampler(3)
# tplan.parent_snode_sampler = GrowingSampler(3)

# CustomRule

In [ ]:
from pkg.planning.task.custom_rules.sweep_entrance_control import SweepEntranceControlRule
tplan.custom_rule = SweepEntranceControlRule(pscene)

## Plan

In [ ]:
from pkg.planning.motion.moveit.moveit_py import ConstrainedSpaceType
mplan.reset_log(False)
gtimer.reset()
# goal_nodes = [("track_face","track_face")+(2,0,0)]
goal_nodes = [("track_face","track_face")+tuple([2 for _ in range(TRACK_NUM)])]
# goal_nodes = [("floor","floor","floor", 2, 2, 2)]
gtimer.tic("plan")
ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, 
              timeout_loop=200, multiprocess=True, timeout=5,  cs_type=ConstrainedSpaceType.TANGENTBUNDLE)
gtimer.toc("plan")
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])

In [ ]:
print(gtimer)

### refine schedule

In [ ]:


# for sweep_ in sweep_list:
#     sweep_.fix_direction = True

# gtimer.tic("refine_trajectory")
# ppline.refine_trajectory(snode_schedule, timeout=1, timeout_constrained=2, N_try=1, max_repeat=3,
#                          plannerconfig=PlannerConfig.RRTstarkConfigDefault,  cs_type=ConstrainedSpaceType.TANGENTBUNDLE,
#                         multiprocess=True)
# gtimer.toc("refine_trajectory")

In [ ]:
# for k,v in ppline.refine_success_dict.items():
#     print("{} {}: {}".format(k, snode_schedule[k].state.node, v))

In [ ]:
# print(gtimer)

## play searched plan

In [ ]:
gscene.clear_highlight()
ppline.play_schedule(snode_schedule[:2], period=0.001)
pscene.set_object_state(initial_state)
gscene.show_pose(crob.home_pose)
time.sleep(0.5)
gscene.show_pose(crob.home_pose)

In [ ]:
ppline.play_schedule(snode_schedule, period=0.05)

## visualize all traj

In [ ]:
gscene.clear_highlight()
effector = gscene.NAME_DICT["brush_face"]
for i_s, snode_to in enumerate(snode_schedule):
    Traj = snode_to.traj
    if Traj is None or (1 in snode_to.state.node[2:]):
        continue
    T_list = effector.draw_traj_coords(Traj, "traj_{}".format(i_s))
#     Z_all = np.array(T_list)[:,2,3]
#     print("std: {}".format(np.std(Z_all)))
#     print("minmax: {}".format(np.max(Z_all)-np.min(Z_all)))

In [ ]:
gscene.clear_highlight()

In [ ]:
print("schedule length: {}".format(len(snode_schedule)))
for i_s,  snode in enumerate(snode_schedule):
#     print("{}: {}".format(i_s, snode.state.node))
    save_json("traj-{}.json".format(i_s), snode.traj)

In [ ]:
traj_list = []
for i_s in range(19):
#     print("{}: {}".format(i_s, snode.state.node))
    traj_list.append(np.array(load_json("traj-{}.json".format(i_s))))

In [ ]:
gscene.show_motion(traj_list[2], period=0.5)

## execute plan

In [ ]:
crob.reset_connection(True,True)

In [ ]:
indy = crob.robot_dict["indy0"]
panda = crob.robot_dict["panda1"]
with indy:
    indy.set_joint_vel_level(1)
    indy.set_task_vel_level(1)

In [ ]:
crob.joint_move_make_sure(crob.home_pose)

In [ ]:
crob.grasp(indy0=False, panda1=False)

In [ ]:
ppline.execute_schedule(snode_schedule, vel_scale=0.5, acc_scale=0.5)